In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

def add_message(input, output):
    memory.save_context(
        {"input": input},
        {"output": output},
    )

add_message(
    "Hi I'm Ben, I live in South Korea",
    "Wow that is so cool!",
)